# USER SIMILARITY BASED RECOMMENDER

Development of a user-based machine learning to recommend books that user still don't read but users with similar caracteres already read and liked them.

The used datasets are abled to download on [link](http://www2.informatik.uni-freiburg.de/~cziegler/BX/)

In [25]:
# libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import NearestNeighbors

In [26]:
# loading data
avaliacoes = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Recommendation Systems/Datasets/BX-Book-Ratings.csv', encoding='latin', sep=';', error_bad_lines=False)
livros =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Recommendation Systems/Datasets/BX-Books.csv', encoding='latin', sep=';', error_bad_lines=False)
usuarios = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Recommendation Systems/Datasets/BX-Users.csv', encoding='latin', sep=';', error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 

## Data Cleaning

In [27]:
usuarios.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [28]:
usuarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [29]:
# cleaning users database
usuarios = usuarios.drop(labels=['Location', 'Age'], axis=1)

In [30]:
livros.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [31]:
livros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [32]:
# cleaning books database
livros = livros.drop(labels=['Year-Of-Publication',	'Publisher',	'Image-URL-S',	'Image-URL-M',	'Image-URL-L'], axis=1)
livros.info()                   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   ISBN         271360 non-null  object
 1   Book-Title   271360 non-null  object
 2   Book-Author  271359 non-null  object
dtypes: object(3)
memory usage: 6.2+ MB


In [33]:
avaliacoes.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


## Pré-Processing

Because of hardware and computational limitations, I reduced the dataframe's lenght the way that it have only users informations that ranked at least 150 books

In [34]:
# reducing the databases's lenght
ids = avaliacoes['User-ID'].value_counts() >= 150
aux = ids[ids].index
avaliacoes = avaliacoes[avaliacoes['User-ID'].isin(aux)]

In [35]:
avaliacoes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 584294 entries, 1456 to 1149637
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   User-ID      584294 non-null  int64 
 1   ISBN         584294 non-null  object
 2   Book-Rating  584294 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 17.8+ MB


In this part, I merge the informations as: ISBN, Book-Title, Book-Author, User-ID and Book-Rating from users, books and ranking turning it into a unique dataframe. As the model is based on users similarity, only the features mentioned above are necessary o modelo é baseado na semelhança entre usuários, apenas os atributos mencionados acima serão necessários. 

In [36]:
# merging cleared databases
livros_avaliacoes = pd.merge(livros, avaliacoes, on='ISBN')
livros_avaliacoes

,ISBN,Book-Title,Book-Author,User-ID,Book-Rating
0,0002005018,Clara Callan,Richard Bruce Wright,11676,8
1,0002005018,Clara Callan,Richard Bruce Wright,85526,0
2,0002005018,Clara Callan,Richard Bruce Wright,96054,0
3,0002005018,Clara Callan,Richard Bruce Wright,177458,0
4,0002005018,Clara Callan,Richard Bruce Wright,263325,6
...,...,...,...,...,...
540794,1582380805,Tropical Rainforests: 230 Species in Full Colo...,"Allen M., Ph.D. Young",275970,0
540795,1845170423,Cocktail Classics,David Biggs,275970,7
540796,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,276680,0
540797,0192126040,Republic (World's Classics),Plato,276680,0


In [37]:
# dropping unecessary features
base_de_dados = livros_avaliacoes.drop(labels=['ISBN', 'Book-Author'], axis=1)
base_de_dados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 540799 entries, 0 to 540798
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Book-Title   540799 non-null  object
 1   User-ID      540799 non-null  int64 
 2   Book-Rating  540799 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 16.5+ MB


In [38]:
base_de_dados.head()

,Book-Title,User-ID,Book-Rating
0,Clara Callan,11676,8
1,Clara Callan,85526,0
2,Clara Callan,96054,0
3,Clara Callan,177458,0
4,Clara Callan,263325,6


This way we can link the User-ID of each user as the table index, the books as features and the rankings as the values.

In [39]:
df_recomendacao = pd.pivot_table(data=base_de_dados, index='User-ID', columns='Book-Title', values='Book-Rating').fillna(0)
df_recomendacao

Book-Title,"A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)",Always Have Popsicles,Apple Magic (The Collector's series),Beyond IBM: Leadership Marketing and Finance for the 1990s,Clifford Visita El Hospital (Clifford El Gran Perro Colorado),Dark Justice,Deceived,"Earth Prayers From around the World: 365 Prayers, Poems, and Invocations for Honoring the Earth",Final Fantasy Anthology: Official Strategy Guide (Brady Games),Flight of Fancy: American Heiresses (Zebra Ballad Romance),...,"Ã?ber den Wunsch, sich wohlzufÃ¼hlen: Geschichten",Ã?coute ma diffÃ©rence (Le Temps des femmes),Ã?ngeles fugaces (Falling Angels),Ã?Â?. Kolumnen.,Ã?Â?ber das Fernsehen.,Ã?Â?ber die Pflicht zum Ungehorsam gegen den Staat.,Ã?Â?lpiraten.,Ã?Â?rger mit Produkt X. Roman.,Ã?Â?stlich der Berge.,Ã?Â?thique en toc
User-ID,,,,,,,,,,,,,,,,,,,,,
254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1733,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1903,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277427,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
277478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
277639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Machine Learning

In [40]:
# k-nearest neighbors
knn_model_user = NearestNeighbors(metric='cosine')

# fitting the model to dataset
knn_model_user.fit(df_recomendacao)

NearestNeighbors(metric='cosine')

In [49]:
import random

# random choice of the user by index
i_user = random.randint(0,len(df_recomendacao))

# getting the users similarity and their index
dist, i_similar_user = knn_model_user.kneighbors(df_recomendacao.iloc[i_user].values.reshape(1,-1), n_neighbors=6)

# getting the user ID
user = df_recomendacao.index[i_user]

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  "X does not have valid feature names, but"


In [50]:
# looking the similar users and each similarity
for i in range(0,len(dist.flatten())):
  if i == 0:
    print(f'USUARIO: {user}\n\n')
  else:
    print(f'similar{i}:{df_recomendacao.index[i_similar_user.flatten()[i]]}, dist: {dist.flatten()[i]}')

USUARIO: 244994


similar1:86189, dist: 0.9249940600879734
similar2:139467, dist: 0.9412158622049625
similar3:168064, dist: 0.9486471270677094
similar4:138097, dist: 0.9490536438994907
similar5:255218, dist: 0.9543509328205353


As the model used is the k-nearest neighbors based on cosine distance. The lowest value of 'dist' displays the closer similarity

In [51]:
# similar-user ID
similar_user = df_recomendacao.index[i_similar_user.flatten()[1]]

In [52]:
# creating user and similar-user dataframes
df_user = df_recomendacao.loc[user].to_frame()
df_similar_user = df_recomendacao.loc[similar_user].to_frame()

In [53]:
# merging the dataframes
df_livros = pd.merge(df_user, df_similar_user, on='Book-Title').sort_values(by=similar_user, ascending=False)
df_livros

,244994,86189
Book-Title,,
Quizzing Canada,0.0,10.0
The Literary Ghost: Great Contemporary Ghost Stories,0.0,10.0
Just an Ordinary Day,0.0,10.0
The Eight,0.0,10.0
The Hours : A Novel,0.0,10.0
...,...,...
Heart of the Country,0.0,0.0
"Heart of the Dragon (Loveswept, No 503)",0.0,0.0
Heart of the Falcon,0.0,0.0


## Recommending to an user

Set a business rule is necessary to make it because we couldn't recommend a book that the user already read. Besides it, the similar-user need to rank the book with a rate at least 7 to recommend it to the user

In [54]:
# Setting business rule
df_livros = df_livros[(df_livros[similar_user]>=7) & (df_livros[user]==0)].reset_index()
df_livros

,Book-Title,244994,86189
0,Quizzing Canada,0.0,10.0
1,The Literary Ghost: Great Contemporary Ghost S...,0.0,10.0
2,Just an Ordinary Day,0.0,10.0
3,The Eight,0.0,10.0
4,The Hours : A Novel,0.0,10.0
...,...,...,...
68,"Ahab's Wife: Or, The Star-Gazer: A Novel",0.0,7.0
69,Stones for Ibarra (Penguin Contemporary Americ...,0.0,7.0
70,Dancing Girls: And Other Stories,0.0,7.0
71,"Two Girls, Fat and Thin",0.0,7.0


In [57]:
# now the dictionary is showing the recommended books to the random user based on his most similar user
books = list(df_livros['Book-Title'][:6])
notas = list(df_livros[similar_user][:6])

recomendar = {'Livro':books,
              'avaliações': notas}
recomendar

{'Livro': ['Quizzing Canada',
  'The Literary Ghost: Great Contemporary Ghost Stories',
  'Just an Ordinary Day',
  'The Eight',
  'The Hours : A Novel',
  'Modern Ghost Stories by Eminent Women Writers'],
 'avaliações': [10.0, 10.0, 10.0, 10.0, 10.0, 10.0]}

In [67]:
print(f'----------------------------------------------------------')
print(f'USER: {user}\n')
print('-----------------------------------------------------------')
print('LIVROS RECOMENDADOS:\n')
for b in range(1,len(books)):
  print(f'\n{books[b]}')
print('-----------------------------------------------------------')

----------------------------------------------------------
USER: 244994

-----------------------------------------------------------
LIVROS RECOMENDADOS:


The Literary Ghost: Great Contemporary Ghost Stories

Just an Ordinary Day

The Eight

The Hours : A Novel

Modern Ghost Stories by Eminent Women Writers
-----------------------------------------------------------
